In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%%capture
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [20]:
import torch
import clip
from PIL import Image
import glob
from IPython.display import HTML
from PIL.Image import open
from torch.cuda import is_available
import glob
import pandas as pd
from itertools import islice
from google.colab import data_table
import re
device = "cuda" if is_available() else "cpu"

In [21]:
def get_model():
    model, preprocess = clip.load('ViT-B/32', device=device)
    return model, preprocess
def atoi(text):
        return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

In [6]:
model, preprocess = get_model()

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 60.6MiB/s]


In [18]:
def get_prediction(frame_path, list_of_labels, how_many_predictions, model, preprocess) -> list:
    Highest3Predictions = []
    name = frame_path.split("/")
    name = name[len(name) -1 ]
    try:
        text = clip.tokenize(list_of_labels).to(device)
        image = preprocess(open(frame_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
            probs = probs.tolist()[0]
        vv = {}
        for i, j in enumerate(probs):
            vv[list_of_labels[i]] = j
        maxx = {k: v for k, v in sorted(vv.items(), key=lambda item: item[1], reverse=True)}
        Highest3Predictions = list(islice(maxx.items(), how_many_predictions))
    except:
        pass
    print(name, Highest3Predictions)
    return Highest3Predictions

In [24]:
def start_process(prompts, folder_path, df):
  frames = []
  for file in glob.glob(folder_path + '/*'):
      frames.append(file)
  frames.sort(key=natural_keys)
  for image_path in frames:
    name = image_path.split("/")
    name = name[len(name) -1 ]
    # df_similarity = get_similarity(image_path, model, preprocess, df_similarity)
    Highest3Predictions = get_prediction(image_path, prompts, 3, model, preprocess)
    c1 = Highest3Predictions[0][0]
    s1 = round(100 * Highest3Predictions[0][1], 2)
    c2 = Highest3Predictions[1][0]
    s2 = round(100 * Highest3Predictions[1][1], 2)
    c3 = Highest3Predictions[2][0]
    s3 = round(100 * Highest3Predictions[2][1], 2)
    cl, fk, score = decodePrediction(c1, c2, c3, s1, s2, s3)
    print(cl, fk)
    print('________________________')
    GOc1 = Highest3Predictions[0][0]
    GOs1 = round((100 * Highest3Predictions[0][1]), 2)
    GOc2 = Highest3Predictions[1][0]
    GOs2 = round((100 * Highest3Predictions[1][1]), 2)
    GOc3 = Highest3Predictions[2][0]
    GOs3 = round((100 * Highest3Predictions[2][1]), 2)
    tmp = [name, cl, GOc1, GOs1, GOc2, GOs2, GOc3, GOs3]
    df_length1 = len(df)
    df.loc[df_length1] = tmp
  return df

In [13]:
def decodePrediction(c1, c2, c3, s1, s2, s3):
    scoreComb3 = s1 + s2 + s3
    scoreComb2 = s1 + s2
    scoreComb1 = s1
    mapped_class_1 = first_level[c1]
    mapped_class_2 = first_level[c2]
    mapped_class_3 = first_level[c3]
    mapped_class = [item for sublist in
                    [mapped_class_1.split(","), mapped_class_2.split(","), mapped_class_3.split(",")] for item in
                    sublist]

    couple = ["image depicting kissing or smooching or romantic lip-lock",
              "image showing couple's close faces, suggesting they are looking each other out of affection",
              "image showing couple's close faces, suggesting they are looking each other out of affection",
              "image showing people or couple hugging each other",
              "image showing a man and a woman sleeping next to each other involving in making out"]

    making_out = ["image depicting a couple holding themselves in a sexual way, indicating making out",
                  "image showing a man and a woman sleeping next to each other involving in making out"]
    classes = [c1, c2, c3]
    if s1 < 5:
        fk = ""
        classification = "neutral"
        score = scoreComb1
    elif mapped_class.count("neutral") >= 3:
        fk = ""
        classification = "neutral"
        score = scoreComb3
    elif mapped_class_1 == "neutral" and mapped_class_2 == "neutral" and scoreComb2 >= 60:
        fk = ""
        classification = "neutral"
        score = scoreComb2
    elif c1 == "image depicting sex toy's like dildos or vibrators" and scoreComb1 >= 30:
        fk = "se_sf_bu_sex_toys"
        classification = "sexuality"
        score = scoreComb1
    elif mapped_class_1 == "pr_fn_ss_middle_finger" and scoreComb1 >= 50:
        fk = "pr_fn_ss_middle_finger"
        classification = "profanity"
        score = scoreComb1
    elif mapped_class.count("pr_fn_ss_middle_finger") >= 2:
        fk = "pr_fn_ss_middle_finger"
        classification = "profanity"
        score = scoreComb2
    elif "image showing face or hands or fingers or legs filling the majority of the frame" in classes and mapped_class.count(
            "neutral") >= 2:
        fk = ""
        classification = "profanity"
        score = scoreComb2
    elif mapped_class_1 == "se_sf_bu_bestiality":
        if scoreComb1 >= 50:
            fk = "se_sf_bu_bestiality"
            classification = "sexuality"
            score = scoreComb1
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb1
    elif mapped_class.count("nudity") == 2 and mapped_class.count("sexuality") == 1:
        classification = "sexuality,nudity"
        fk = ""
        score = scoreComb3
    elif mapped_class.count("sexuality") == 2 and mapped_class.count("nudity") == 1:
        classification = "sexuality"
        fk = ""
        score = scoreComb3
    elif mapped_class_1 == "neutral" and c2 in making_out and c3 in making_out:
        classification = "sexuality"
        fk = ""
        score = scoreComb2
    elif c1 == "image showing a man and a woman sitting on a couch and talking to each other" and c3 == "image showing a naked man and a naked woman sleeping or sitting next to each other, indicating nudity":
        classification = "nudity"
        fk = ""
        score = scoreComb2
    elif c1 == "image showing a completely naked man and a completely naked woman sleeping or sitting next to each other, indicating nudity" and s1 >= 10:
        classification = "nudity"
        fk = ""
        score = scoreComb1
    elif c3 == "image showing a completely naked man and a completely naked woman sleeping or sitting next to each other, indicating nudity" and mapped_class.count(
            "neutral") == 2:
        classification = "nudity"
        fk = ""
        score = s3
    elif c1 == "image showing couple's close faces, suggesting they are looking each other out of affection" and c2 == "image depicting a couple holding themselves in a sexual way, indicating making out" and c3 == "image showing people or couple hugging each other":
        classification = "sexuality"
        fk = ""
        score = scoreComb3
    elif c1 == "image showing people or couple hugging each other" and (
            mapped_class_2 == "sexuality" or mapped_class_3 == "sexuality"):
        classification = "sexuality"
        fk = ""
        score = scoreComb3
    elif c1 == "image showing people or couple hugging each other" and (
            mapped_class_2 == "nudity" or mapped_class_3 == "nudity"):
        classification = "nudity"
        fk = ""
        score = scoreComb3
    elif c1 == "image showing people or couple hugging each other" and (
            mapped_class_2 == "sexuality,nudity" or mapped_class_3 == "sexuality,nudity"):
        classification = "sexuality,nudity"
        fk = ""
        score = scoreComb3
    elif c1 == "image depicting a couple holding themselves in a sexual way, indicating making out" and c2 in couple and c3 in couple:
        classification = "sexuality"
        fk = ""
        score = scoreComb3
    elif mapped_class_1 == "violence,sexuality" and s1 >= 40:
        classification = "violence,sexuality"
        score = s1
        fk = ""
    elif mapped_class_1 == "violence,sexuality" and s1 >= 25:
        classification = "violence"
        score = s1
        fk = ""
    elif mapped_class_1 == "alcohol,drugs" and s1 >= 60:
        classification = "alcohol,drugs"
        score = s1
        fk = ""
    elif mapped_class_1 == "alcohol,drugs" and s1 >= 25:
        classification = "alcohol"
        score = s1
        fk = ""
    elif mapped_class_1 == "smoking,drugs" and s1 >= 35:
        classification = "smoking,drugs"
        score = s1
        fk = ""
    elif mapped_class_1 == "smoking,drugs" and s1 >= 25:
        classification = "smoking"
        score = s1
        fk = ""
    elif mapped_class_1 == "smoking,alcohol" and s1 >= 25:
        classification = "smoking,alcohol"
        score = s1
        fk = ""
    elif mapped_class_1 == "smoking,nudity" and s1 >= 30:
        classification = "smoking,nudity"
        score = s1
        fk = ""
    elif mapped_class_1 == "drugs,nudity" and s1 >= 45:
        classification = "drugs,nudity"
        score = s1
        fk = ""
    elif mapped_class_1 == "drugs,nudity" and s1 >= 35:
        classification = "nudity"
        score = s1
        fk = ""
    elif mapped_class_1 == "alcohol,violence" and s1 >= 25:
        classification = "alcohol,violence"
        score = s1
        fk = ""
    elif mapped_class_1 == "violence,nudity" and s1 >= 25:
        classification = "violence,nudity"
        score = s1
        fk = ""
    elif mapped_class_1 == "sexuality,nudity" and s1 >= 30:
        classification = "sexuality,nudity"
        score = s1
        fk = ""
    elif mapped_class_1 == "alcohol,nudity" and s1 >= 20:
        classification = "alcohol,nudity"
        score = s1
        fk = ""
    elif mapped_class.count("sexuality") >= 2:
        tt = s2 + s3
        if s1 <= 10:
            fk = ""
            classification = "neutral"
            score = scoreComb3
        elif c1 == "image showing couple's close faces, suggesting they are looking each other out of affection" and s1 > 40:
            fk = ""
            classification = "neutral"
            score = scoreComb2
        elif c1 == "image showing couple's close faces, suggesting they are looking each other out of affection" and s1 > 20 and c2 == "image depicting kissing or smooching or romantic lip-lock":
            fk = ""
            classification = "neutral"
            score = scoreComb2
        elif mapped_class_2 == "nudity" or mapped_class_2 == "sexuality,nudity":
            fk = ""
            classification = "sexuality,nudity"
            score = scoreComb3
        elif mapped_class_3 == "nudity":
            fk = ""
            classification = "sexuality"
            score = scoreComb3
        elif mapped_class_1 == "neutral" and tt >= 25:
            fk = ""
            classification = "sexuality"
            score = scoreComb3
        elif mapped_class_1 == "sexuality" and scoreComb3 >= 50:
            fk = ""
            classification = "sexuality"
            score = scoreComb3
        elif mapped_class_3 == "neutral" and scoreComb2 >= 40:
            fk = ""
            classification = "sexuality"
            score = scoreComb3
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb3
    elif mapped_class_1 == "alcohol" and s1 >= 30:
        classification = "alcohol"
        score = s1
        fk = ""
    elif mapped_class_1 == "nudity" and s1 >= 30:
        classification = "nudity"
        score = s1
        fk = ""
    elif mapped_class_1 == "sexuality" and s1 >= 30 and c1 != "image showing couple's close faces, suggesting they are looking each other out of affection":
        classification = "sexuality"
        score = s1
        fk = ""
    elif mapped_class_1 == "violence" and s1 >= 30:
        classification = "violence"
        score = s1
        fk = ""
    elif mapped_class_1 == "smoking" and s1 >= 30:
        classification = "smoking"
        score = s1
        fk = ""
    elif mapped_class_1 == "drugs" and s1 >= 30:
        classification = "drugs"
        score = s1
        fk = ""
    elif mapped_class.count("sexuality") >= 3 and scoreComb3 >= 30:
        classification = "sexuality"
        score = scoreComb3
        fk = ""
    elif mapped_class.count("nudity") >= 3 and scoreComb3 >= 25:
        fk = ""
        classification = "nudity"
        score = scoreComb3
    elif (mapped_class.count("violence") >= 3 and scoreComb3 >= 30):
        fk = ""
        classification = "violence"
        score = scoreComb3
    elif mapped_class.count("drugs") >= 3 and scoreComb3 >= 30:
        fk = ""
        classification = "drugs"
        score = scoreComb3
    elif mapped_class.count("alcohol") >= 3 and scoreComb3 >= 30:
        fk = ""
        classification = "alcohol"
        score = scoreComb3
    elif mapped_class.count("smoking") >= 3 and scoreComb3 >= 30:
        fk = ""
        classification = "smoking"
        score = scoreComb3
    elif mapped_class_1 == "nudity" and mapped_class_2 == "nudity" and scoreComb2 >= 40:
        fk = ""
        classification = "nudity"
        score = scoreComb2
    elif mapped_class_1 == "violence" and mapped_class_2 == "violence" and scoreComb2 >= 40:
        fk = ""
        classification = "violence"
        score = scoreComb2
    elif mapped_class_1 == "drugs" and mapped_class_2 == "drugs" and scoreComb2 >= 40:
        fk = ""
        classification = "drugs"
        score = scoreComb2
    elif mapped_class_1 == "alcohol" and mapped_class_2 == "alcohol" and scoreComb2 >= 40:
        fk = ""
        classification = "alcohol"
        score = scoreComb2
    elif mapped_class_1 == "smoking" and mapped_class_2 == "smoking" and scoreComb2 >= 40:
        fk = ""
        classification = "smoking"
        score = scoreComb2
    elif mapped_class.count("drugs") == 2 and scoreComb3 >= 50:
        fk = ""
        classification = "drugs"
        score = scoreComb3
    elif mapped_class.count("nudity") >= 2 and mapped_class.count("neutral") >= 1:
        tt = s2 + s3
        if mapped_class_1 == "neutral" and tt >= 25:
            fk = ""
            classification = "nudity"
            score = scoreComb3
        elif mapped_class_1 == "nudity" and scoreComb3 >= 50:
            fk = ""
            classification = "nudity"
            score = scoreComb3
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb3
    elif mapped_class.count("violence") >= 2 and mapped_class.count("neutral") >= 1:
        tt = s2 + s3
        if mapped_class_1 == "neutral" and tt >= 25:
            fk = ""
            classification = "violence"
            score = scoreComb3
        elif mapped_class_1 == "violence" and scoreComb3 >= 50:
            fk = ""
            classification = "violence"
            score = scoreComb3
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb3
    elif mapped_class.count("drugs") >= 2 and mapped_class.count("neutral") >= 1:
        tt = s2 + s3
        if mapped_class_1 == "neutral" and tt >= 25:
            fk = ""
            classification = "drugs"
            score = scoreComb3
        elif mapped_class_1 == "drugs" and scoreComb3 >= 50:
            fk = ""
            classification = "drugs"
            score = scoreComb3
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb3
    elif mapped_class.count("alcohol") >= 2 and mapped_class.count("neutral") >= 1:
        tt = s2 + s3
        if mapped_class_1 == "neutral" and tt >= 25:
            fk = ""
            classification = "alcohol"
            score = scoreComb3
        elif mapped_class_1 == "alcohol" and scoreComb3 >= 50:
            fk = ""
            classification = "alcohol"
            score = scoreComb3
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb3
    elif mapped_class.count("smoking") >= 2 and mapped_class.count("neutral") >= 1:
        tt = s2 + s3
        if mapped_class_1 == "neutral" and tt >= 25:
            fk = ""
            classification = "smoking"
            score = scoreComb3
        elif mapped_class_1 == "smoking" and scoreComb3 >= 50:
            fk = ""
            classification = "smoking"
            score = scoreComb3
        else:
            fk = ""
            classification = "neutral"
            score = scoreComb3
    else:
        fk = ""
        classification = "neutral"
        score = scoreComb1

    return classification, fk, score

In [25]:
first_level = {
    # Nothing on the screen black blank:
    "image showing an empty or unoccupied space": "neutral",
    "image showing a still or blank or uneventful background or black background": "neutral",
    "image depicting frame with little visibility or blur background": "neutral",
    "image showing passive or monotonous or insignificant event": "neutral",
    "image out of focus or blurry due to motion or camera shake": "neutral",
    "image depicting unrecognized activity": "neutral",
    "image depicting a close-up shot of an unrecognized object": "neutral",

    # Logo, warning, credits, object:
    "image containing centered words or credits on a screen": "neutral",
    "image containing statutory warning": "neutral",
    "image showing a movie production company or movie studio logo or movie title still": "neutral",
    "image containing a brand logo or service logo or trademark logo or organization logo or a product logo": "neutral",
    "image depicting an object filling majority of the frame": "neutral",
    "image showing the use of a device or a gadget": "neutral",
    "image showing any objects that are typically used on a regular basis or household objects": "neutral",
    "image showing movement of automobiles or vehicles": "neutral",
    "image showing water bottles or fruit juice bottles or water containers or cola tins": "neutral",
    "image showing face or hands or fingers or legs filling the majority of the frame": "neutral",
    "image depicting cropped face or cropped hands or cropped legs or cropped body": "neutral",

    # People, Group, Person:
    "image containing gathering of group of people, people roaming around": "neutral",

    # Shots
    "image depicting a person's body filling majority of the frame": "neutral",

    # Animal:
    "image depicting animal roaming freely in its natural habitat": "neutral",
    "image depicting a human petting or holding an animal affectionately": "neutral",
    "image depicting humans playing with animals": "neutral",
    "image depicting animal enjoying themselves": "neutral",
    "image depicting animal mating or mating habit": "neutral",

    # People Activity:
    "image depicting people engaged in a particular sports activity": "neutral",
    "image depicting discussion between group of people": "neutral",
    "image containing a human that is a woman, a girl, a man, a boy not engaged in particular activity": "neutral",
    "image depicting group of people engaged in a particular neutral activity": "neutral",
    "image showing people or couple hugging each other": "neutral",
    "image showing couple or people sleeping or resting": "neutral",
    "image showing people sleeping in the dark or at night": "neutral",
    "image showing people walking": "neutral",
    "image depicting people gesturing": "neutral",
    "image depicting joyful people dancing, enjoying themselves": "neutral",
    "image showing people driving a vehicle": "neutral",
    "image showing people inside a store or shop": "neutral",
    "image showing people in a park or outdoor setting": "neutral",
    "image showing people in a healthcare or professional setting": "neutral",
    "image showing people exercising": "neutral",
    "image showing people using public transportation": "neutral",
    "image showing people holding a child or baby": "neutral",
    "image showing teenagers enjoying themselves on the street": "neutral",
    "image showing people engaged in a hobby or pastime": "neutral",
    "image showing people performing a work-related activity": "neutral",
    "image showing people performing a leisure or relaxation activity": "neutral",
    "image showing people performing a everyday or routine activity": "neutral",
    "image showing people playing musical instruments and singing": "neutral",
    "image depicting people sitting around and enjoying their company": "neutral",
    "image showing a man and a woman standing next to each other and looking around": "neutral",
    "image showing a man and a woman sitting on a couch and talking to each other": "neutral",
    "image showing a man and a woman sleeping next to each other and talking": "neutral",
    "image showing a couple sitting very close to each other, indicating their affection": "neutral",
    "image showing couple's close faces, suggesting they are looking each other out of affection": "neutral",
    "image showing conversation between teenagers on a street corner": "neutral",
    "image showing a person clicking photo using the camera": "neutral",
    "image showing people in bikini and underwear enjoying themselves": "neutral",
    "image showing a woman dressed in a bra and panties or lingerie": "neutral",
    "image showing a cropped image of a beautiful woman, with a focus on the face": "neutral",

    # Expressions:
    "image depicting emotions or feeling expressing through their facial expression": "neutral",
    "image depicting affection in a friendly or familial context": "neutral",
    "image showing facial or lips expressions indicating pouting or mouth movement": "neutral",
    "image depicting signs of tiredness such as drooping eyelids or slouching posture": "neutral",

    # Other:
    "image showing a woman is sitting, talking, looking around": "neutral",
    "image showing a man is sitting, talking, looking around": "neutral",

    # Settings:
    "image showing a natural landscape": "neutral",
    "image showing an urban setting": "neutral",
    "image showing a building or architecture": "neutral",
    "image showing a sunset or sunrise": "neutral",
    "image showing waterfall or river": "neutral",
    "image showing a plant or tree": "neutral",
    "image showing a night sky": "neutral",
    "image showing a beach or ocean": "neutral",
    "image showing a desert or mountainous terrain": "neutral",
    "image showing a forest or jungle": "neutral",
    "image showing a snowy or icy environment or icy floor": "neutral",
    "image showing a city skyline": "neutral",
    "image showing a traffic scene or highway": "neutral",
    "image showing a body of water or lake": "neutral",
    "image showing a building interior or interior design": "neutral",
    "image showing a neutral outdoor scene or natural setting": "neutral",

    # Violence:
    "image depicting animal cruelty or neglect or abuse or assault": "violence",
    "image depicting animals being experimented on in scientific research": "violence",
    "image depicting animals being killed or harmed": "violence",
    "image depicting blood or gore in the context of animal violence or hunting": "violence",

    # Weapon shown
    "image depicting firearms or weapons being displayed": "violence",
    "image showing a weapon used for violence kept on the surface": "violence",
    "image depicting bladed weapons visible in the image, such as knives, swords, or machetes": "violence",
    "image depicting firearms weapons visible in the image, such as a handgun, rifle, or shotgun": "violence",
    "image depicting long-range weapons visible in the image, such as crossbows or sniper rifles": "violence",
    "image containing any vehicles or machinery that could be used as weapons, such as tanks, bulldozers, or drones": "violence",
    "image containing any explosive devices, such as bombs or grenades": "violence",
    "image depicting guns are fired or brandished": "violence",
    "image depicting showing a gun for threatening": "violence",
    "image depicting people armed with weapons": "violence",
    "image showing the assembly or production of firearms": "violence",
    "image showing the transaction of firearms or other weapons": "violence",
    "image depicting violent act of stabbing knife in the body": "violence",
    "image depicting brandishing knives or other sharp objects as weapons": "violence",
    "image depicting a bomb explodes in a crowded marketplace, causing chaos and destruction": "violence",
    "image depicting people are shown using heavy artillery to destroy an enemy stronghold, with buildings crumbling "
    "and explosions everywhere": "violence",

    # Blood Gore:
    "image depicting blood that has been shed, especially as a result of violence": "violence",
    "image depicting violence or injuries resulting in severe blood loss": "violence",
    "image containing any bloodstains or pools of blood on the ground or other surfaces": "violence",
    "image depicting people or animals bleeding or hemorrhaging": "violence",
    "image depicting violence leading to significant blood loss or gore": "violence",
    "image depicting surgical procedures or medical interventions that involve blood or other bodily fluid": "violence",
    "image depicting visible wounds or injuries in the image, such as cuts, bruises, or abrasions": "violence",
    "image depicting any acts of physical violence, such as punching, kicking, or choking, that could result in bleeding or injury": "violence",
    "image depicting any instances of self-harm or suicide that involve blood in this image": "violence",

    # Victim State:
    "image depicting any instances of individuals who have suffered fatal injuries or trauma, such as gunshot wounds or severe head injurie": "violence",
    "image depicting the lifeless human bodies were piled, indicating violent act": "violence",
    "image depicting human remains that have been scattered or otherwise disturbed, suggesting violence or disruption": "violence",
    "image depicting bodies that appear to have been dismembered or disfigured, such as missing limbs or severe facial injuries": "violence",
    "image containing any instances of bodies that have been burned or charred beyond recognition": "violence",
    "image depicting any instances of bodies that have been intentionally mutilated, such as through cutting or carving": "violence",
    "image depicting bodies that have been decomposed or otherwise altered after death, such as by animals or insects": "violence",
    "image depicting any instances of torture or other extreme violence that have resulted in mutilation": "violence",
    "image depicting any instances of medical procedures or autopsies that involve mutilated bodies or body parts": "violence",
    "image depicting individuals who appear to be deceased, such as lying motionless or in an unnatural position": "violence",

    # Violent Act:
    "image depicting heavily bleeding person is shown lying on the ground, indicating violent acts": "violence",
    "image depicting violent acts of physical assault or battery": "violence",
    "image depicting the victims brutally assaulted, indicating violent act": "violence",
    "image depicting the victim tied behind their backs or strapped their head and legs restrained, indicating "
    "violent act of executions": "violence",
    "image depicting violent act of decapitating and executions": "violence",
    "image depicting a group of people are shown firing their weapons in a battle, explosions and gunfire filling the "
    "air, indicating violent acts": "violence",
    "image depicting violent acts of torture or threats, indicating a victim is tied up or held in captivity": "violence",
    "image depicting two people engage in a sword fight, clanging their blades together and dodging each other's "
    "attacks, indicating violent acts": "violence",
    "image depicting someone is stabbing other person with a sharp knife, indicating violent acts": "violence",

    # Fighting
    "image depicting group of people engage in a brutal fistfight, kicking and punching each other on the ground": "violence",

    # Other:
    "image depicting violent acts of bombings or explosives, indicating explosion aftermath": "violence",

    # Drug Shown:
    "image depicting cocaine powder packet, cocaine powder or powder lines kept on the surface, indicating drug shown": "drugs",
    "image depicting someone holding a cocaine powder substance in a packet or pouch, indicating drug shown": "drugs",
    "image depicting holding a small bag of marijuana, with them proceeding to roll a joint, indicating drug use": "drugs",
    "image depicting snorting cocaine off of a table, with the powder shown, indicating drug use": "drugs",
    "image depicting a person is holding a pill bottle and popping or swallowing several pills, indicating drug "
    "consumption": "drugs",
    "image depicting prescription pills, indicating drug shown": "drugs",
    "image depicting a bag of white powder is shown on a table, with someone snorting it through a rolled up bill, "
    "indicating drug use": "drugs",
    "image depicting purchasing heroin from a drug dealer on a street corner shows the exchange of money for drugs": "drugs",

    "image depicting psychedelics drug tablets, indicating drug shown": "drugs",
    "image showing illicit consumption of marijuana or cannabis or marijuana joint, indicating drug use": "drugs",
    "image showing prescription pill bottle, pills or tablets, indicating drug shown": "drugs",
    "image depicting syringe or black band used for injecting heroin, indicating drug use": "drugs",

    # Manufacturing:
    "image depicting drug production and manufacturing process": "drugs",
    "image depicting a setup for manufacturing illicit drugs": "drugs",
    "image depicting people packing illegal drugs in a packet, indicating drug manufacturing assembly": "drugs",

    # Preparation Consumption:
    "image depicting injecting a needle, indicating drug use": "drugs",
    "image showing drug paraphernalia or equipment used to consume drugs": "drugs",
    "image depicting a group of people huddled around a table, counting piles of white powder, syringes and pills": "drugs",
    "image depicting someone is shown using a lighter to heat up a substance on a spoon before injecting it": "drugs",
    "image depicting a line of white powder is shown being snorted off a flat surface by a character": "drugs",
    "image depicting a baggie of drugs is shown indicating drugs transaction": "drugs",
    "image depicting a character is shown cutting a white powder into lines with a razor blade before snorting it": "drugs",

    # Smoking
    # Use:
    "image depicting smoking cigarette by holding cigarette in mouth": "smoking",
    "image showing individuals smoking or holding a lit cigarette in their hand": "smoking",
    "image depicting a person, cigarette burning in his hand or mouth, indicating cigarette smoking": "smoking",
    "image containing any depictions of tobacco products, including cigarettes or cigars, that might suggest tobacco use": "smoking",
    "image depicting lots of cigarettes or cigarette packet kept on the surface": "smoking",
    "image depicting several ashtrays filled to the brim with cigarette butts, evidence of the chain-smoking habits": "smoking",
    "image depicting someone lighting up a cigarette, indicating cigarette smoking": "smoking",
    "image showing individuals who appear to be smoking, even if the cigarette is not clearly visible": "smoking",
    "image showing glimpse of cigarettes, indicating smoking": "smoking",
    # Alcohol
    # Shown
    "image showing beer bottles or beer cans kept on the surface, indicating alcohol shown": "alcohol",
    "image depicting alcohol cabinets and shelving for liquor storage": "alcohol",
    "image depicting a crowded bar, showing patrons drinking and laughing, with glasses and bottles of alcohol on "
    "every table": "alcohol",
    "image showing a person taking a sip of whiskey straight from the bottle, indicating alcohol use": "alcohol",
    "image showing glasses of red and white wine being consumed or offered ": "alcohol",
    "image depicting people say cheers over a liquor, indicating alcohol use": "alcohol",
    "image depicting implied alcohol use": "alcohol",
    "image depicting lots of liquor bottles and glasses indicating alcohol addiction": "alcohol",
    # Use:
    "image depicting people consuming alcohol beverages or alcoholic drinks": "alcohol",
    "image depicting people sitting around and enjoying drinking alcohol beverages or alcoholic drinks": "alcohol",
    "image depicting clearly visible alcohol bottle in the corner": "alcohol",
    "image depicting a person in a bikini or underwear drinking alcohol with bottle or glass": "alcohol",
    "image showing people in bikini and underwear drinking alcohol": "alcohol",

    # Sexuality
    "image depicting sexual acts that is considered as bestiality": "sexuality",

    # Intercourse:
    "image depicting sexual act of vaginal sex or penetrative sex, indicating sexual intercourse": "sexuality,nudity",
    "image depicting blow job on penis or licking genitals, vagina or sucking cock, indicating a sexual act involving genitals": "sexuality",
    "image depicting a sexual act of insertion and thrusting of the penis into the vagina for sexual intercourse": "sexuality,nudity",

    # Sex Toy
    "image depicting sex toy's like dildos or vibrators": "sexuality",

    # Making Out, Kissing
    "image depicting a couple holding themselves in a sexual way, indicating making out": "sexuality",
    "image showing a man and a woman sleeping next to each other involving in making out": "sexuality",
    "image depicting a nude couple involved in making out, indicating an erotic sexual act": "sexuality",
    "image depicting kissing or smooching or romantic lip-lock": "sexuality",
    "image showing sensual passionate playful couple kissing making love, young seductive sexy attractive woman on "
    "top wearing black bra embracing touching arousing tempting man lover enjoying foreplay together": "sexuality",
    "image containing any instances of sexually suggestive poses or gestures, such as individuals touching or "
    "caressing their own body or another person's body": "sexuality",

    # BDSM Masturbation
    "image depicting sexual act of masturbation, jerking the penis or fingering the vagina": "sexuality",
    "image depicting erotic practices of engaging in fetishistic behaviors, such as bondage, BDSM, or other kinks": "sexuality",

    # Group Sex
    "image depicting group sex or orgy": "sexuality",

    # Nudity Views:
    "image depicting naked baby showing genitals": "nudity",
    "image depicting a person showing the full naked back body including bare back and ass or buttocks": "nudity",
    "image depicting a female showing her full naked frontal body including breast, nipples, vagina or vulva": "nudity",
    "image depicting a male showing his full naked frontal body including nipples, penis or dick": "nudity",
    "image depicting a person showing the lower naked body, with focus on buttocks or ass": "nudity",
    "image depicting a male showing his upper naked body, with focus on bare chest": "nudity",
    "image depicting a female showing her upper naked body, with focus on nipples and breast": "nudity",
    "image depicting groups of the fully naked crowd on the field or in the room": "nudity",
    "image showing clearly visible exposed genitals": "nudity",
    "image showing close-up shot of private parts, indicating nudity": "nudity",
    "image depicting a naked woman, with a focus on the completely naked body": "nudity",
    "image showing a completely naked man and a completely naked woman sleeping or sitting next to each other, indicating nudity": "nudity",

    # Non Human
    "image depicting a painting of fully naked body including breast and nipples and genitals": "nudity",
    "image depicting a naked blow up doll, a type of anthropomorphic sex toy": "nudity",
    "image depicting sculptures that resemble genitalia or other sexualized body parts": "nudity",
    "image depicting statues that depict nudity or intimate body parts": "nudity",
    "image depicting mannequins that are shown without clothing or that appear to be displaying intimate body parts": "nudity",

    # Clothes
    "image showing a person wearing clothing specifically designed for swimming or water activities": "neutral",
    "image showing a nude man wearing briefs or underwear exposing chest and nipples": "nudity",
    "image showing a gorgeous seductive woman lying completely naked on bed, indicating nudity": "nudity",
    "image showing a woman dressed in a revealing clothes or exposing dress and her private parts are visible from the dress": "nudity",
    "image depicting clothes worn by the person exposing a lot of skin or body parts": "nudity",
    "image depicting naked people enjoying themselves exposing their private body parts": "nudity",

    # profanity
    "image depicting gestures using the middle finger": "pr_fn_ss_middle_finger",
    "image depicting the middle finger being used in combination with profanity": "pr_fn_ss_middle_finger",

    # combination
    "image depicting signs of sexual violence or sexual assault": "violence,sexuality",
    "image depicting liquor bottles and other drug paraphernalia kept on the surface": "alcohol,drugs",
    "image depicting the pile of discarded needles, cigarette butts, and other drug paraphernalia scattered on the "
    "floor, indicating drug use": "smoking,drugs",
    "image depicting a group of people smoking cigarettes and white powder lines are visible on the surface": "smoking,drugs",
    "image depicting people sit around a table at a bar, drinks in hand and cigarettes burning in ashtrays, "
    "smoking cigarettes": "smoking,alcohol",
    "image depicting alcohol bottles shown and cigarette shown indicating drinking alcohol and smoking cigarettes": "smoking,alcohol",
    "image depicting nude people or person smoking cigarettes": "smoking,nudity",
    "image depicting weapon used or weapon shown in a liquor store": "alcohol,violence",
    "image depicting alcohol bottles shown behind a violent act": "alcohol,violence",
    "image depicting weapon kept on a table along with beer or whiskey bottles or glass": "alcohol,violence",
    "image depicting the nude person or people covered with blood or gore": "violence,nudity",
    "image depicting a gored and bloody naked body, showing deep lacerations and stab wounds": "violence,nudity",

}

In [26]:
df = pd.DataFrame(columns=["image", "cl", "c1", "s1", "c2", "s2", "c3", "s3"])
folder_path = "/content/gdrive/MyDrive/frames"
df = start_process(list(first_level.keys()), folder_path, df)

Streaming output truncated to the last 5000 lines.
sexuality 
________________________
597_img_000008.png [('image depicting kissing or smooching or romantic lip-lock', 0.46337890625), ('image depicting a couple holding themselves in a sexual way, indicating making out', 0.141357421875), ("image showing couple's close faces, suggesting they are looking each other out of affection", 0.101806640625)]
sexuality 
________________________
598_img_000004.png [('image depicting a nude couple involved in making out, indicating an erotic sexual act', 0.26953125), ('image depicting a couple holding themselves in a sexual way, indicating making out', 0.26123046875), ('image showing people or couple hugging each other', 0.055633544921875)]
sexuality 
________________________
598_img_000008.png [('image depicting a couple holding themselves in a sexual way, indicating making out', 0.2237548828125), ('image showing a man and a woman sleeping next to each other involving in making out', 0.10083007812

In [1]:
df

NameError: ignored

In [ ]:
df.to_csv("/content/gdrive/MyDrive/output/predictions.csv")